# 测试hdf5

In [5]:
import pandas as pd
import gzip
from pandas import HDFStore
import numpy as np
import os

In [43]:
def analyze_zinc_pdbqt_gz(pdbqt_gz_path: str):
    coor = []
    index = []
    with gzip.open(pdbqt_gz_path, 'rb') as f:
        t_start = 0
        t_end = 0
        for line in f:
            if line.startswith(b'ATOM'):
                t_end += 1
                coor.append([str(line[13:16].strip(), 'utf-8'), float(line[30:38]), float(line[38:46]), float(line[46:54])])
            if line.startswith(b'REMARK  Name = '):
                if t_end == 0:
                    zinc_id = str(line[15:].strip(), 'utf-8')
                    continue
                index.append([zinc_id, t_start, t_end])  # 存储上一个分子的信息
                zinc_id = str(line[15:].strip(), 'utf-8')  # 当前分子的id
                t_start = t_end
        index.append([zinc_id, t_start, t_end])
    return pd.DataFrame(coor, columns=['atom', 'x', 'y', 'z']), pd.DataFrame(index, columns=['zinc_id', 'start', 'end']).set_index('zinc_id', drop=True)


In [44]:
with HDFStore('data.h5') as store:
    for path, sub_dir, filename in os.walk('BJ'):
        for file in filename:
            if file.endswith('.pdbqt.gz'):
                # print(file)
                d = os.path.join(path, file[:-13])
                # store.put(os.path.join(path, file[:-13]), analyze_zinc_pdbqt_gz(os.path.join(path, file)))
                store[d+'_coor'], store[d+'_index'] = analyze_zinc_pdbqt_gz(os.path.join(path, file))


BJAAMN.xaa.pdbqt.gz
BJAAMO.xaa.pdbqt.gz
BJAAMP.xaa.pdbqt.gz
BJAARM.xaa.pdbqt.gz
BJAARN.xaa.pdbqt.gz
BJAARO.xaa.pdbqt.gz
BJAARP.xaa.pdbqt.gz
BJABMM.xaa.pdbqt.gz
BJABMN.xaa.pdbqt.gz
BJABMO.xaa.pdbqt.gz
BJABRM.xaa.pdbqt.gz
BJABRN.xaa.pdbqt.gz
BJABRO.xaa.pdbqt.gz
BJADMO.xaa.pdbqt.gz
BJADRN.xaa.pdbqt.gz
BJADRO.xaa.pdbqt.gz
BJBAMM.xaa.pdbqt.gz
BJBARL.xaa.pdbqt.gz
BJBARN.xaa.pdbqt.gz
BJBBMN.xaa.pdbqt.gz
BJBBRM.xaa.pdbqt.gz
BJBBRN.xaa.pdbqt.gz
BJBDRN.xaa.pdbqt.gz
BJCARN.xaa.pdbqt.gz
BJCBRN.xaa.pdbqt.gz
BJCBRO.xaa.pdbqt.gz
BJCDRN.xaa.pdbqt.gz
BJEAMO.xaa.pdbqt.gz
BJEARM.xaa.pdbqt.gz
BJEARN.xaa.pdbqt.gz
BJEARO.xaa.pdbqt.gz
BJEBMM.xaa.pdbqt.gz
BJEBMN.xaa.pdbqt.gz
BJEBMO.xaa.pdbqt.gz
BJEBRM.xaa.pdbqt.gz
BJEBRN.xaa.pdbqt.gz
BJEBRO.xaa.pdbqt.gz
BJEDMO.xaa.pdbqt.gz
BJEDRN.xaa.pdbqt.gz
BJEDRO.xaa.pdbqt.gz


In [27]:
analyze_zinc_pdbqt_gz(os.path.join(path, file))

(    atom      x      y      z
 0      N  3.914 -1.411  1.133
 1      H  4.081 -0.960  2.019
 2      H  3.606 -0.726  0.459
 3      C  2.878 -2.440  1.294
 4      C  3.288 -3.405  2.409
 ..   ...    ...    ...    ...
 955    C -0.910 -2.034  0.892
 956    C -1.141 -1.168  2.132
 957    C -0.891 -1.146 -0.354
 958    C -1.676 -1.899 -2.540
 959    C -0.786 -2.010 -1.584
 
 [960 rows x 4 columns],
                   start  end
 zinc_id                     
 ZINC000132004333      0   19
 ZINC000386807859     19   37
 ZINC000386807858     37   56
 ZINC000133105987     56   75
 ZINC000133106195     75   93
 ZINC000133106566     93  111
 ZINC000133106406    111  129
 ZINC000132001328    129  147
 ZINC000132000715    147  165
 ZINC000132000926    165  183
 ZINC000132001127    183  201
 ZINC000393938770    201  219
 ZINC000393938768    219  238
 ZINC000132003731    238  257
 ZINC000132004136    257  275
 ZINC000132003904    275  293
 ZINC000310147526    293  311
 ZINC000310147525    311  330
 

In [47]:
store = HDFStore('data.h5')
k = store.keys()
print(k[0], k[1])
# store[k[0]], store[k[1]]
c = store[k[0]]
i = store[k[1]]
c[i['start'][0]:i['end'][0]]
# store['/BJ/EDRO/BJEDRO']
# store.info()

/BJ/EDRO/BJEDRO_coor /BJ/EDRO/BJEDRO_index


,atom,x,y,z
0,N,3.914,-1.411,1.133
1,H,4.081,-0.960,2.019
2,H,3.606,-0.726,0.459
3,C,2.878,-2.440,1.294
4,C,3.288,-3.405,2.409
5,C,2.281,-4.556,2.482
6,C,1.550,-1.774,1.660
7,C,1.078,-0.904,0.493
8,C,-0.350,1.055,0.781
9,C,-0.230,-0.247,0.853


In [42]:
# store.root.BJAAMN
store.close()